<a href="https://colab.research.google.com/github/tententgc/se-model-txt-prediction/blob/main/Marong_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [1]:
!pip install datasets --quiet
!pip install pythainlp --quiet
!pip install transformers --quiet
!pip install sentencepiece accelerate --quiet
!pip install accelerate -U --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 100.4 MB/s eta 0:00:00


In [41]:
!pip install evaluate --quiet

In [42]:
import evaluate

In [43]:
# data preprocessing
import pythainlp
import re
import numpy as np
import pandas as pd


# model
import torch
from torch import nn
import pickle
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSequenceClassification,
    AutoConfig,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    pipeline
)

from datasets import (
    Dataset,
    DatasetDict,
    Features, Sequence, ClassLabel, Value,
    load_dataset
)

In [3]:
!pip show datasets

Name: datasets
Version: 3.2.0
Summary: HuggingFace community-driven open-source library of datasets
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, dill, filelock, fsspec, huggingface-hub, multiprocess, numpy, packaging, pandas, pyarrow, pyyaml, requests, tqdm, xxhash
Required-by: 


## Import data

In [8]:
!mkdir dataset

In [9]:
!wget "https://raw.githubusercontent.com/nlp-chula/traffy-predict/main/dataset/TF-checked-train-dataset-v2.csv" -O dataset/train.csv
!wget "https://raw.githubusercontent.com/nlp-chula/traffy-predict/main/dataset/TF-checked-test-dataset-v2.csv" -O dataset/test.csv

--2024-12-27 20:54:50--  https://raw.githubusercontent.com/nlp-chula/traffy-predict/main/dataset/TF-checked-train-dataset-v2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 701514 (685K) [text/plain]
Saving to: ‘dataset/train.csv’

dataset/train.csv   100%[===================>] 685.07K  --.-KB/s    in 0.005s  

2024-12-27 20:54:51 (125 MB/s) - ‘dataset/train.csv’ saved [701514/701514]

--2024-12-27 20:54:51--  https://raw.githubusercontent.com/nlp-chula/traffy-predict/main/dataset/TF-checked-test-dataset-v2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awa

In [10]:
df = pd.read_csv("/content/dataset/train.csv")
test_df = pd.read_csv("/content/dataset/test.csv")

## Data Preprocessing

In [11]:
train_df, dev_df = train_test_split(df, test_size=0.2, random_state=42)

In [12]:
print(len(train_df))
print(len(dev_df))
print(len(test_df))

1744
436
546


In [13]:
labels = ['ต้นไม้',
 "('ทางเท้า', 'รถจอดขวางทางเท้า')",
 "('ไฟฟ้า', 'เสาไฟฟ้า&สายไฟฟ้า')",
 'ประปา',
 "('ทางเท้า', 'ท่อระบายน้ำบนทางเท้า')",
 'สัตว์',
 'อุปกรณ์ชำรุด',
 'น้ำท่วม',
 "('ทางเท้า', 'หาบเร่แผงลอย')",
 'reject',
 'รถสาธารณะ',
 'กรุงเทพโปร่งใส&ทุจริต',
 'จราจร',
 'วินมอเตอร์ไซค์',
 "('ถนน', 'ท่อระบายน้ำบนถนน')",
 "('ถนน', 'รถจอดขวางถนน')",
 'คนไร้บ้าน&ขอทาน',
 "('ไฟฟ้า', 'ไฟส่องสว่าง')",
 "('ถนน', 'พื้นผิวถนนชำรุด')",
 'อาคารสถานที่ชำรุด',
 'มลภาวะทางอากาศ',
 "('ขยะ', 'น้ำเน่าเสีย')",
 'สวนสาธารณะ',
 'เสียง',
 "('ทางเท้า', 'ทางเท้าชำรุด')",
 "('ขยะ', 'ขยะ')",
 "('ถนน', 'ทางม้าลาย')",
 'การเข้าถึงของคนพิการ&ผู้สูงอายุ']

In [14]:
train_df['description'] = train_df['description'].apply(lambda x: x.replace('|', ''))
dev_df['description'] = dev_df['description'].apply(lambda x: x.replace('|', ''))
test_df['description'] = test_df['description'].apply(lambda x: x.replace('|', ''))

In [15]:
label_to_idx = {label:i for i,label in enumerate(labels)}

train_df['label'] = train_df['label'].apply(lambda x: label_to_idx[x])
dev_df['label'] = dev_df['label'].apply(lambda x: label_to_idx[x])
test_df['label'] = test_df['label'].apply(lambda x: label_to_idx[x])

label_list = [0] * len(label_to_idx)
for label in label_to_idx:
  label_list[label_to_idx[label]] = label

## Model

In [16]:
model_name = 'clicknext/phayathaibert'

In [17]:
config = AutoConfig.from_pretrained(model_name, num_labels=len(labels))
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           config=config)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at clicknext/phayathaibert and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
train_df = train_df.reset_index(drop=True)
dev_df = dev_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [19]:
features= Features({
                    "description": Value(dtype='string'),
                    "label": ClassLabel(names=label_list)
                })
d = DatasetDict({'train': Dataset.from_pandas(train_df, features=features),
                 'dev': Dataset.from_pandas(dev_df, features=features),
                 'test': Dataset.from_pandas(test_df, features=features)})

In [20]:
d

DatasetDict({
    train: Dataset({
        features: ['description', 'label'],
        num_rows: 1744
    })
    dev: Dataset({
        features: ['description', 'label'],
        num_rows: 436
    })
    test: Dataset({
        features: ['description', 'label'],
        num_rows: 546
    })
})

In [21]:
d['train'][0]

{'description': '1. การจราจรจากบางกะปิ มุ่งหน้า5แยกลาดพร้าว บริเวณแยก รัชดา-ลาดพร้าว เลนซ้ายสุดที่จะไปเส้นรัชดามุ่งหน้าอโศก รถเมล์ รถส่วนตัว จอดขวางรอไฟเขียวตรงมุ่งหน้า5แยกลาดพร้าว เวลาไฟเขียวเลี้ยวซ้ายแล้วไม่สามารถไปได้ อยากให้มีเจ้าหน้าที่ช่วยกวดขันชั่วโมงเร่งด่วน กรณีใกล้เคียงแยกวังหินเลนซ้ายผ่านตลอดแต่ก็มีรถเลนตรงมาจอดขวางในชั่วโมงเร่งด่วน',
 'label': 12}

In [22]:
def tokenize(examples):
    tokenized_inputs = tokenizer(examples["description"],
                                 is_split_into_words=False,
                                 truncation=True,
                                 max_length=100)
    return tokenized_inputs
tokenized_datasets = d.map(tokenize, batched=True)

Map:   0%|          | 0/1744 [00:00<?, ? examples/s]

Map:   0%|          | 0/436 [00:00<?, ? examples/s]

Map:   0%|          | 0/546 [00:00<?, ? examples/s]

In [23]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['description', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1744
    })
    dev: Dataset({
        features: ['description', 'label', 'input_ids', 'attention_mask'],
        num_rows: 436
    })
    test: Dataset({
        features: ['description', 'label', 'input_ids', 'attention_mask'],
        num_rows: 546
    })
})

In [24]:
tokenized_datasets['train'][0]['input_ids']

[5,
 3428,
 10,
 3948,
 32,
 19837,
 10,
 5770,
 65,
 818,
 7216,
 10,
 22679,
 10,
 13012,
 35,
 7216,
 10,
 2303,
 2027,
 329,
 9625,
 841,
 13012,
 5770,
 15967,
 10,
 5071,
 10,
 180,
 744,
 10,
 2373,
 4933,
 289,
 15375,
 704,
 5770,
 65,
 818,
 7216,
 3731,
 15375,
 17278,
 22,
 363,
 2342,
 4422,
 10585,
 6920,
 19697,
 797,
 3709,
 7199,
 2285,
 818,
 1619,
 1672,
 2303,
 2027,
 199,
 530,
 7852,
 180,
 2303,
 704,
 13737,
 4933,
 20,
 797,
 3709,
 6]

In [25]:
tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][0]['input_ids'])

['<s>',
 '▁1.',
 '▁',
 'การจราจร',
 'จาก',
 'บางกะปิ',
 '▁',
 'มุ่งหน้า',
 '5',
 'แยก',
 'ลาดพร้าว',
 '▁',
 'บริเวณแยก',
 '▁',
 'รัชดา',
 '-',
 'ลาดพร้าว',
 '▁',
 'เลน',
 'ซ้าย',
 'สุด',
 'ที่จะไป',
 'เส้น',
 'รัชดา',
 'มุ่งหน้า',
 'อโศก',
 '▁',
 'รถเมล์',
 '▁',
 'รถ',
 'ส่วนตัว',
 '▁',
 'จอด',
 'ขวาง',
 'รอ',
 'ไฟเขียว',
 'ตรง',
 'มุ่งหน้า',
 '5',
 'แยก',
 'ลาดพร้าว',
 '▁เวลา',
 'ไฟเขียว',
 'เลี้ยวซ้าย',
 'แล้ว',
 'ไม่สามารถ',
 'ไปได้',
 '▁อยากให้',
 'มีเจ้าหน้า',
 'ที่ช่วย',
 'กวดขัน',
 'ชั่วโมง',
 'เร่งด่วน',
 '▁กรณี',
 'ใกล้เคียง',
 'แยก',
 'วัง',
 'หิน',
 'เลน',
 'ซ้าย',
 'ผ่าน',
 'ตลอด',
 'แต่ก็มี',
 'รถ',
 'เลน',
 'ตรง',
 'มาจอด',
 'ขวาง',
 'ใน',
 'ชั่วโมง',
 'เร่งด่วน',
 '</s>']

In [26]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['description', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1744
    })
    dev: Dataset({
        features: ['description', 'label', 'input_ids', 'attention_mask'],
        num_rows: 436
    })
    test: Dataset({
        features: ['description', 'label', 'input_ids', 'attention_mask'],
        num_rows: 546
    })
})

In [27]:
data_collator = DataCollatorWithPadding(tokenizer,
                                        padding=True,
                                        pad_to_multiple_of=8)
tokenized_datasets["train"].column_names

['description', 'label', 'input_ids', 'attention_mask']

In [28]:
tokenized_datasets['dev'][20]

{'description': 'อยากให้ช่วยตัดแต่งต้นไม้',
 'label': 0,
 'input_ids': [5, 4422, 189, 600, 1159, 2414, 6],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [29]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k != 'description'}
[len(x) for x in samples["input_ids"]]

[72, 14, 52, 18, 11, 23, 22, 39]

In [30]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 72]),
 'attention_mask': torch.Size([8, 72]),
 'labels': torch.Size([8])}

In [45]:
args = TrainingArguments(
    output_dir=model_name,
    learning_rate=5e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=30,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    evaluation_strategy='steps',
    eval_steps=25,
    logging_steps=25,
    save_strategy='steps'
)

def compute_metrics(eval_preds):
  metric = evaluate.load("accuracy")
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-45-e41f4616323a>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


### Model Training

In [46]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
25,0.778700,0.938702,0.743119
50,0.635500,0.855353,0.761468
75,0.401400,0.784773,0.791284
100,0.283000,0.819012,0.795872
125,0.175700,0.816985,0.793578
150,0.137500,0.866081,0.793578
175,0.094200,0.855807,0.807339
200,0.062800,0.874134,0.805046
225,0.047300,0.941414,0.793578
250,0.033400,1.020859,0.793578


Could not locate the best model at clicknext/phayathaibert/checkpoint-175/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


TrainOutput(global_step=840, training_loss=0.08508210414133612, metrics={'train_runtime': 919.5848, 'train_samples_per_second': 56.895, 'train_steps_per_second': 0.913, 'total_flos': 2259598886903808.0, 'train_loss': 0.08508210414133612, 'epoch': 30.0})

In [47]:
trainer.state.log_history

[{'loss': 0.7787,
  'grad_norm': 4.143901824951172,
  'learning_rate': 4.8511904761904764e-05,
  'epoch': 0.8928571428571429,
  'step': 25},
 {'eval_loss': 0.9387015700340271,
  'eval_accuracy': 0.7431192660550459,
  'eval_runtime': 3.5213,
  'eval_samples_per_second': 123.818,
  'eval_steps_per_second': 1.988,
  'epoch': 0.8928571428571429,
  'step': 25},
 {'loss': 0.6355,
  'grad_norm': 3.4662156105041504,
  'learning_rate': 4.7023809523809525e-05,
  'epoch': 1.7857142857142856,
  'step': 50},
 {'eval_loss': 0.8553534150123596,
  'eval_accuracy': 0.7614678899082569,
  'eval_runtime': 3.074,
  'eval_samples_per_second': 141.834,
  'eval_steps_per_second': 2.277,
  'epoch': 1.7857142857142856,
  'step': 50},
 {'loss': 0.4014,
  'grad_norm': 3.6840438842773438,
  'learning_rate': 4.5535714285714286e-05,
  'epoch': 2.678571428571429,
  'step': 75},
 {'eval_loss': 0.7847728133201599,
  'eval_accuracy': 0.7912844036697247,
  'eval_runtime': 3.1312,
  'eval_samples_per_second': 139.245,
  '

### Model Prediction

In [48]:
predictions = trainer.predict(tokenized_datasets['test'])
preds = np.argmax(predictions.predictions, axis=-1)

In [49]:
predictions

PredictionOutput(predictions=array([[-1.8009758 ,  0.88118297,  0.3940311 , ..., -1.6380907 ,
         0.29685265, -1.0417296 ],
       [-0.30523074, -0.5558791 , -0.6638932 , ...,  0.3785733 ,
        -0.6440258 ,  0.4720932 ],
       [-0.90033144, -0.67674303, -1.5807276 , ..., -1.1940634 ,
         0.49447882, -1.1701545 ],
       ...,
       [-0.51520437, -0.5965246 , -0.98297155, ...,  0.55377406,
        -1.0963734 , -0.38794452],
       [ 0.9323996 , -0.95629895, -0.26779884, ...,  2.2815347 ,
        -0.60402036,  0.13504615],
       [-1.5371363 , -0.5828512 , -1.3406914 , ..., -2.0282154 ,
        -0.6428914 , -1.1384333 ]], dtype=float32), label_ids=array([17, 16, 15, 25, 16, 25,  4, 25, 17,  6,  9, 26, 14, 15, 23, 25,  5,
       25, 25, 26,  6,  5, 14, 22, 20, 20, 10,  7, 25, 21, 12,  0, 14, 22,
        5,  9,  4, 27, 16, 14, 24,  4,  9,  9, 22,  4, 10,  9,  9, 25, 10,
       26, 22,  0, 17,  4, 18, 27,  0,  6,  6, 25,  7, 21,  7, 14, 15, 11,
        7, 13, 27, 24,  7, 25,  

In [50]:
predictions.predictions

array([[-1.8009758 ,  0.88118297,  0.3940311 , ..., -1.6380907 ,
         0.29685265, -1.0417296 ],
       [-0.30523074, -0.5558791 , -0.6638932 , ...,  0.3785733 ,
        -0.6440258 ,  0.4720932 ],
       [-0.90033144, -0.67674303, -1.5807276 , ..., -1.1940634 ,
         0.49447882, -1.1701545 ],
       ...,
       [-0.51520437, -0.5965246 , -0.98297155, ...,  0.55377406,
        -1.0963734 , -0.38794452],
       [ 0.9323996 , -0.95629895, -0.26779884, ...,  2.2815347 ,
        -0.60402036,  0.13504615],
       [-1.5371363 , -0.5828512 , -1.3406914 , ..., -2.0282154 ,
        -0.6428914 , -1.1384333 ]], dtype=float32)

In [51]:
predicted_labels = [labels[x] for x in preds]
gold = [labels[x] for x in predictions.label_ids]

### Test-topk-accuracy

In [52]:


test_results = predictions
labels_id = test_results.label_ids

y_pred_pt = torch.from_numpy(test_results.predictions)
probs = nn.functional.softmax(y_pred_pt, dim=-1)

# Get top-k probabilities and indices (let's say k=3 for example)
k = 3
topk_probabilities, topk_indices = torch.topk(probs, k, dim=1)

In [53]:
def topk_compute(k, probs):
  topk_probabilities, topk_indices = torch.topk(probs, k, dim=1)
  topk_pred_lst = [[labels[id] for id in sublist] for sublist in topk_indices.numpy()]

  eval_data = {"gold-label": gold, "topk-pred":topk_pred_lst}
  eval_df = pd.DataFrame(eval_data)

  deemed_correct = [1 if gold in pred else 0 for pred, gold in zip(eval_df['topk-pred'], eval_df['gold-label'])]
  eval_df['deemed-correct'] = deemed_correct

  topk_acc = len(eval_df[eval_df['deemed-correct'] == 1])/len(eval_df)

  print(f"Top {k} accuracy is {topk_acc}")

In [54]:
for i in range(1, 8):
  topk_compute(i, probs)


Top 1 accuracy is 0.8095238095238095
Top 2 accuracy is 0.8901098901098901
Top 3 accuracy is 0.9212454212454212
Top 4 accuracy is 0.9358974358974359
Top 5 accuracy is 0.9432234432234432
Top 6 accuracy is 0.945054945054945
Top 7 accuracy is 0.9468864468864469


### Test-1-accuracy

In [55]:
preds = np.argmax(predictions.predictions, axis=-1)

In [56]:
print(classification_report(gold, predicted_labels))

                                     precision    recall  f1-score   support

                     ('ขยะ', 'ขยะ')       0.96      0.82      0.88        28
             ('ขยะ', 'น้ำเน่าเสีย')       0.82      0.75      0.78        12
               ('ถนน', 'ทางม้าลาย')       0.84      1.00      0.91        32
        ('ถนน', 'ท่อระบายน้ำบนถนน')       0.50      0.50      0.50        12
         ('ถนน', 'พื้นผิวถนนชำรุด')       0.62      0.85      0.72        27
            ('ถนน', 'รถจอดขวางถนน')       0.40      0.50      0.44        16
        ('ทางเท้า', 'ทางเท้าชำรุด')       0.82      0.58      0.68        24
('ทางเท้า', 'ท่อระบายน้ำบนทางเท้า')       0.68      0.71      0.70        21
    ('ทางเท้า', 'รถจอดขวางทางเท้า')       0.75      0.92      0.83        13
        ('ทางเท้า', 'หาบเร่แผงลอย')       0.57      0.67      0.62         6
     ('ไฟฟ้า', 'เสาไฟฟ้า&สายไฟฟ้า')       0.60      0.60      0.60         5
           ('ไฟฟ้า', 'ไฟส่องสว่าง')       0.88      0.79      0.83        1

### Top-1-accuracy with super label

In [57]:
pattern = r"\('([ก-์]+)'"
super_gold = [re.match(pattern, label).group(1) if re.match(pattern, label) else label for label in gold]
super_pred_labels = [re.match(pattern, label).group(1) if re.match(pattern, label) else label for label in predicted_labels]


In [58]:
print(classification_report(super_gold, super_pred_labels))

                                 precision    recall  f1-score   support

                         reject       0.91      0.79      0.85        38
          กรุงเทพโปร่งใส&ทุจริต       0.89      0.62      0.73        13
การเข้าถึงของคนพิการ&ผู้สูงอายุ       0.88      0.92      0.90        24
                            ขยะ       0.94      0.82      0.88        40
                คนไร้บ้าน&ขอทาน       0.97      1.00      0.99        37
                          จราจร       0.57      0.54      0.55        24
                         ต้นไม้       0.93      1.00      0.96        41
                            ถนน       0.67      0.83      0.74        87
                        ทางเท้า       0.79      0.77      0.78        64
                        น้ำท่วม       0.86      0.57      0.69        21
                          ประปา       0.67      0.77      0.71        13
                 มลภาวะทางอากาศ       0.80      0.89      0.84        18
                      รถสาธารณะ       0.93      0.

# Save model

In [61]:
trainer.save_model("./model-marong")

In [62]:
tokenizer.save_pretrained("./model-marong")

('./model-marong/tokenizer_config.json',
 './model-marong/special_tokens_map.json',
 './model-marong/sentencepiece.bpe.model',
 './model-marong/added_tokens.json',
 './model-marong/tokenizer.json')

In [72]:
labels

['ต้นไม้',
 "('ทางเท้า', 'รถจอดขวางทางเท้า')",
 "('ไฟฟ้า', 'เสาไฟฟ้า&สายไฟฟ้า')",
 'ประปา',
 "('ทางเท้า', 'ท่อระบายน้ำบนทางเท้า')",
 'สัตว์',
 'อุปกรณ์ชำรุด',
 'น้ำท่วม',
 "('ทางเท้า', 'หาบเร่แผงลอย')",
 'reject',
 'รถสาธารณะ',
 'กรุงเทพโปร่งใส&ทุจริต',
 'จราจร',
 'วินมอเตอร์ไซค์',
 "('ถนน', 'ท่อระบายน้ำบนถนน')",
 "('ถนน', 'รถจอดขวางถนน')",
 'คนไร้บ้าน&ขอทาน',
 "('ไฟฟ้า', 'ไฟส่องสว่าง')",
 "('ถนน', 'พื้นผิวถนนชำรุด')",
 'อาคารสถานที่ชำรุด',
 'มลภาวะทางอากาศ',
 "('ขยะ', 'น้ำเน่าเสีย')",
 'สวนสาธารณะ',
 'เสียง',
 "('ทางเท้า', 'ทางเท้าชำรุด')",
 "('ขยะ', 'ขยะ')",
 "('ถนน', 'ทางม้าลาย')",
 'การเข้าถึงของคนพิการ&ผู้สูงอายุ']

# Model simulation

In [73]:
model_name = "/content/model-marong"

In [74]:
input_text = 'อันตรายมาก ทางข้ามถนนโรงเรียน สงสารเด็ก'

## Test model output

In [75]:
model = pipeline(model=model_name,
                    tokenizer=model_name,
                    task="text-classification")

Device set to use cuda:0


In [76]:
result = model(input_text, top_k=4)

In [79]:
labels = ['ต้นไม้',
 "('ทางเท้า', 'รถจอดขวางทางเท้า')",
 "('ไฟฟ้า', 'เสาไฟฟ้า&สายไฟฟ้า')",
 'ประปา',
 "('ทางเท้า', 'ท่อระบายน้ำบนทางเท้า')",
 'สัตว์',
 'อุปกรณ์ชำรุด',
 'น้ำท่วม',
 "('ทางเท้า', 'หาบเร่แผงลอย')",
 'reject',
 'รถสาธารณะ',
 'กรุงเทพโปร่งใส&ทุจริต',
 'จราจร',
 'วินมอเตอร์ไซค์',
 "('ถนน', 'ท่อระบายน้ำบนถนน')",
 "('ถนน', 'รถจอดขวางถนน')",
 'คนไร้บ้าน&ขอทาน',
 "('ไฟฟ้า', 'ไฟส่องสว่าง')",
 "('ถนน', 'พื้นผิวถนนชำรุด')",
 'อาคารสถานที่ชำรุด',
 'มลภาวะทางอากาศ',
 "('ขยะ', 'น้ำเน่าเสีย')",
 'สวนสาธารณะ',
 'เสียง',
 "('ทางเท้า', 'ทางเท้าชำรุด')",
 "('ขยะ', 'ขยะ')",
 "('ถนน', 'ทางม้าลาย')",
 'การเข้าถึงของคนพิการ&ผู้สูงอายุ']

In [84]:
predicted_label_index = int(result[0]['label'].split("_")[1])
predicted_label = labels[predicted_label_index]
print(predicted_label)

('ถนน', 'ทางม้าลาย')


# Save the model

In [86]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [88]:
!cp -r /content/model-marong /content/drive/MyDrive/Marong